In [ ]:
# write a program to demonstrate the change in accuracy/loss/convergence time with
# change in optimizers like stochastic gradient descent,adam, adagrad, RMSprop and
# nadam for any suitable application

In [ ]:
import tensorflow as tf
import time
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

In [ ]:
# Load and preprocess the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize pixel values

In [ ]:
# Convert labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [ ]:
# Define a function to build a CNN model
def build_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        Flatten(),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])
    return model

In [ ]:
# Define the optimizers to compare
optimizers = {
    "SGD": tf.keras.optimizers.SGD(),
    "Adam": tf.keras.optimizers.Adam(),
    "Adagrad": tf.keras.optimizers.Adagrad(),
    "RMSprop": tf.keras.optimizers.RMSprop(),
    "Nadam": tf.keras.optimizers.Nadam()
}


In [ ]:
# Train and evaluate the model with each optimizer, and track the results
results = {}

for opt_name, optimizer in optimizers.items():
    print(f"\nTraining with {opt_name} optimizer:")

    # Rebuild the model for each optimizer
    model = build_model()
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    # Record start time
    start_time = time.time()

    # Train the model
    history = model.fit(x_train, y_train, epochs=10, batch_size=64, validation_data=(x_test, y_test), verbose=2)

    # Record end time and calculate duration
    duration = time.time() - start_time

    # Evaluate on test set
    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)

    # Store results
    results[opt_name] = {
        "history": history,
        "test_loss": test_loss,
        "test_accuracy": test_acc,
        "convergence_time": duration
    }


In [ ]:
 # Plot accuracy and loss for each optimizer
plt.figure(figsize=(14, 6))

# Plot accuracy
plt.subplot(1, 2, 1)
for opt_name, result in results.items():
    plt.plot(result["history"].history["accuracy"], label=f'{opt_name} Train')
    plt.plot(result["history"].history["val_accuracy"], linestyle='--', label=f'{opt_name} Val')
plt.title("Model Accuracy per Epoch")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()


# Plot loss
plt.subplot(1, 2, 2)
for opt_name, result in results.items():
    plt.plot(result["history"].history["loss"], label=f'{opt_name} Train')
    plt.plot(result["history"].history["val_loss"], linestyle='--', label=f'{opt_name} Val')
plt.title("Model Loss per Epoch")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

plt.tight_layout()
plt.show()



In [ ]:
# Display convergence times and final accuracy/loss for each optimizer
for opt_name, result in results.items():
    print(f"\n{opt_name} Optimizer:")
    print(f"  - Test Accuracy: {result['test_accuracy']:.4f}")
    print(f"  - Test Loss: {result['test_loss']:.4f}")
    print(f"  - Convergence Time: {result['convergence_time']:.2f} seconds")